In [1]:
print('[-] Importing packages...')
# File Connection and File Manipulation
import os
import pickle
import gzip
import json
# Basic Data Science Toolkits
import pandas as pd
import numpy as np
import math
import random
import time
import re
import itertools
import datetime
# Basic Data Vizualization
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
# Text Preprocessing(other)
import string
import nltk

[-] Importing packages...


In [2]:
print('[-] Importing packages...')
try:
  import pyvis
  from pyvis.network import Network
except:
  !pip install pyvis
  import pyvis
  from pyvis.network import Network

[-] Importing packages...


In [3]:
#Download required corpus based data to nltk package
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("averaged_perceptron_tagger")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\silve\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\silve\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\silve\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\silve\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
gDEBUG = True

In [5]:
#see the GPU assigned
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Apr  1 16:43:54 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.52                 Driver Version: 551.52         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   58C    P5             18W /  115W |    2664MiB /   8192MiB |     27%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
ROOT_DIR = "./"
DATA_DIR = "%s/data" % ROOT_DIR
EVAL_DIR = "%s/evaluation" % ROOT_DIR
MODEL_DIR = "%s/models" % ROOT_DIR

#Create missing directories, if they don't exist
if not os.path.exists(DATA_DIR):
    # Create a new directory because it does not exist
    os.makedirs(DATA_DIR)
    print("The data directory is created!")
if not os.path.exists(EVAL_DIR):
    # Create a new directory because it does not exist
    os.makedirs(EVAL_DIR)
    print("The evaluation directory is created!")
if not os.path.exists(MODEL_DIR):
    # Create a new directory because it does not exist
    os.makedirs(MODEL_DIR)
    print("The model directory is created!")

The data directory is created!
The evaluation directory is created!
The model directory is created!


In [7]:
#link would be expired by the time of the grading. It's just here to show how it could be done.
#import urllib.request
#url = 'https://d3c33hcgiwev3.cloudfront.net/Rylguh2DR_CpYLodg0fwQQ_1199ff1bd63f4163bc4de1e7806c70f1_nikelululemonadidas_tweets.jsonl.gz?Expires=1712102400&Signature=i9uv4cAI6bDpP2tP3nODY~qmBSPEt~ywYmy6H0i9CIjG2siek56an3vNbEzUC8BESaVtb-sjqQAALUtjHeyTEANAs8dnUXs~vISoeH7FydUKZYO5rJAjO2Xj~NxYRBt1e78rUILA4ZPP2LWWbrvxpOBW-yNBWu19KAH8nk74DrM_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A'
#filename = './data/nikelululemonadidas_tweets.jsonl.gz'
#urllib.request.urlretrieve(url, filename)

In [13]:
LIMIT = 5
tweet_file_path = '%s/nikelululemonadidas_tweets.jsonl.gz' % DATA_DIR
with gzip.open(tweet_file_path) as f:
    for i, line in enumerate(f):
        if i >= LIMIT:
            break
        tweet = json.loads(line)
        text = tweet.get("full_text") or tweet.get("text")
        if "nike" in text.lower():
            print('tweet', i,':', text, '\n')

tweet 0 : #ad The Nike Women's Air More Uptempo 96 'White/Opti Yellow' is now available via @footlocker! |$160| #SneakerScouts @Nike https://t.co/5lAq7b2ffU https://t.co/wmjxIcsheP 

tweet 2 : Proof @LaserShip is stealing. I work from home and have a ring doorbell. @wsoctv @Nike @wcnc @wbtv @bbb_us https://t.co/9o3stezjgs 

tweet 3 : RT @pyleaks: *LEAK ALERT*: The next Supreme x @Nike collab for Spring 2022 will feature the Nike Shox Ride 2.
The duo will be dropping 3 co… 

tweet 4 : RT @SneakerScouts: #ad The Space Jam x Nike LeBron 18 Low 'Sylvester vs. Tweety' is now available via @snipes_usa! |$160| #SneakerScouts @K… 



In [14]:
tweet

{'created_at': 'Fri Oct 01 06:22:08 +0000 2021',
 'id': 1443823529296867300,
 'id_str': '1443823529296867350',
 'full_text': "RT @SneakerScouts: #ad The Space Jam x Nike LeBron 18 Low 'Sylvester vs. Tweety' is now available via @snipes_usa! |$160| #SneakerScouts @K…",
 'truncated': False,
 'display_text_range': [0, 140],
 'entities': {'hashtags': [{'text': 'ad', 'indices': [19, 22]},
   {'text': 'SneakerScouts', 'indices': [122, 136]}],
  'symbols': [],
  'user_mentions': [{'screen_name': 'SneakerScouts',
    'name': '#SneakerScouts',
    'id': 792182745942233100,
    'id_str': '792182745942233089',
    'indices': [3, 17]},
   {'screen_name': 'snipes_usa',
    'name': 'SNIPES_USA',
    'id': 83606701,
    'id_str': '83606701',
    'indices': [102, 113]}],
  'urls': []},
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 